# Imaginary time evolution

$
\require{physics}
\def\bm{\boldsymbol}
$

In [ ]:
import numpy as np
from scipy.linalg import expm

from ph121c_lxvm import tensor, models

## What we are doing

In this assignment there will be simulations of the dynamics of quantum systems
with local Hamiltonians in the matrix product state (MPS) representation.
This will be via the Time Evolving Block Decimation (TEBD) algorithm.

If you're like me and you're learning this for the first time, I looked for help
on the topic and have created this brief collection of learning materials:

- [Vidal's exposition of TEBD (2004)](https://arxiv.org/abs/quant-ph/0310089)
- [Tensor Network TEBD](https://tensornetwork.org/mps/algorithms/timeevo/tebd.html)
- [TeNPy (see `literature` for more)](https://tenpy.github.io/index.html)
- [Help with tensor networks](https://www.tensors.net/)
- [White's exposition of DMRG (1992)](https://doi.org/10.1103/PhysRevLett.69.2863)

## Overview

In TEBD, we are interested in applying a propagator $U(t)$ to evolve a quantum
state. In this section, we will use imaginary time evolution of the Hamiltonian:
$$ U(\tau) = e^{H \tau}, $$
but next section we will consider real time evolution:
$$ U(t) = e^{-i H t}. $$
We will use the following TFIM Hamiltonian with open boundary conditions
parametrized by $h^x, h^z$:
$$
    H = 
        -\sum_{i=1}^{L-1} \sigma_i^z \sigma_{i+1}^z
        -h^x \sum_{i=1}^L \sigma_i^x 
        -h^z \sum_{i=1}^L \sigma_i^z 
    .
$$
Because the Hamiltonian is 2-local, we can group its terms:
\begin{align}
    H &=
        -\left( \sum_{i=1}^L h^x \sigma_i^x + h^z \sigma_i^z \right)
        - \left( \sum_{i=1}^{L // 2} \sigma_{2i - 1}^z \sigma_{2i}^z \right)
        - \left( \sum_{i=1}^{L // 2-1+L\%2} \sigma_{2i}^z \sigma_{2i+1}^z \right)
    \\\\
        &= H_1 + H_2^\text{even} + H_2^\text{odd}
        .
\end{align}
Now all of the terms in each group commute, but the groups themselves may not.
The Zassenhaus formula [(proven here)](https://doi.org/10.1002/cpa.3160070404)
allows a separation of $U(t)$ into a product of matrix exponentials of these
local terms, grouped in powers of $t$.
The formula tells us that the lowest order of $t$ in the exponent is given by:
$$
    U(\tau) =
    e^{H_1 \tau} e^{H_2^\text{even} \tau} e^{H_2^\text{odd} \tau}
    e^{\mathcal O (\tau^2)}
    .
$$
Note: the Zassenhaus formula looks like the Baker-Campbell-Hausdorff formula,
but they group terms differently. The former groups terms in powers of $t$, and
thus lends itself to perturbative approximations.

A corollary of the Zassenhaus formula is the Lie product formula:
$$
    e^{A + B} = \lim_{N \to \infty} \left( e^{A/N} e^{B/N} \right)^N
    .
$$
In fact, this result was known as early as 1893 by Sophus Lie, the namesake of
Lie algebras! Their work is published in ISBN 0828402329 and on the
[web](https://archive.org/details/theoriedertrans00liegoog?ref=ol&view=theater).
Thus for some finite $N$, we will are in a good position to make the 
[Suzuki](https://aip.scitation.org/doi/abs/10.1063/1.529425)-[Trotter
](https://doi.org/10.1090/S0002-9939-1959-0108732-6) decomposition
which provides an approximation of the time-evolution operator over a discrete
number of time steps:
$$
    U(\tau) \approx
        \left(
        e^{H_1 \tau/N} e^{H_2^\text{even} \tau/N} e^{H_2^\text{odd} \tau/N}
        \right)^N
    .
$$
This provides yet another example of how mathematicians have explored areas
relevant to physics more than a century before their emergence.

In practice, **this is how we will simulate time evolution of quantum systems**
on both classical computers (using the MPS representation) _and_ on quantum
computers by applying the gates on qudits in the same fashion as on the tensor
network. For higher-order Suzuki-Trotter decompositions,
[read this (cited by Vitali)](https://arxiv.org/abs/quant-ph/9809009).

## Matrix calculus

I haven't given you enough details yet to apply this to a tensor network!
I haven't exponentiated matrices that can be written as Kronecker products.

In general, matrix exponentials can be calculated in terms of diagonalizing a
matrix and exponentiating its eigenvalues, or by a power-series representation.
For a review of matrix exponentials, see these [course notes written by my summer
research mentor](https://web.mit.edu/18.06/www/Spring17/Matrix-Exponentials.pdf),
or this [paper on many ways of calculating the matrix exponential
](http://www.cs.cornell.edu/cvResearchPDF/19ways+.pdf).

My questions about Kronecker products and matrix exponentials center around
whether the operations commute: for 1-site operators, 2-site operators?
Let's begin

In [ ]:
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.diag([1, -1])

### Two sites
For the diagonal matrix $\sigma^z$ we shall see that the matrix exponential and
Kronecker product do not commute for a 2-site operator:

In [ ]:
expm(np.kron(sz, sz))

In [ ]:
np.kron(expm(sz), expm(sz))

Not the same! This means we actually have to exponentiate the matrix, which for
a diagonal matrix is equivalent to exponentiating the diagonal:

In [ ]:
np.exp(np.diag(np.kron(sz, sz)))

The consequence for MPS is that this is not possible to represent as a one-site
operator, so we will have to contract a virtual index before applying this
transformation directly. (After, we will again do an SVD to return to MPS form).

We _could_ represent the matrix product operator as acting on individual sites
if we take an SVD of it and disaggregating the physical indicies by introducing
a virtual index. This option might not be viable for diagonal matrices with
large coefficients (maybe better for random unitaries):

In [ ]:
np.linalg.svd(expm(np.kron(sz, sz)))

That is, if we can write the elements of a 2-site operator $U(t)$ as
$U_{\sigma'_i \sigma'_{i+1}}^{\sigma_i \sigma_{i+1}}$, then we should reshape
the matrix so that $U_{\sigma'_i \sigma_i}^{\sigma'_{i+1} \sigma_{i+1}}$
groups the physical indices by the site. Then we should do an SVD on this matrix
which will introduce a virtual index $\alpha$, leading to:
$$
    U_{\sigma'_i \sigma_i}^{\sigma'_{i+1} \sigma_{i+1}}
        = \sum_\alpha U_{\sigma'_i \sigma_i}^\alpha
        S_\alpha^\alpha (V^\dagger)_{\alpha}^{\sigma'_{i+1} \sigma_{i+1}}
    .
$$
We should then reshape
$U_{\sigma'_i \sigma_i}^\alpha \to U_{\sigma'_i}^{\sigma_i \alpha}$ and
$(V^\dagger)_{\alpha}^{\sigma'_{i+1} \sigma_{i+1}}
\to (V^\dagger)_{\alpha \sigma'_{i+1}}^{\sigma_{i+1}}$
so that we can apply these operators to the physical indices as a matrix
multiplication.

### One site

#### Diagonal matrix

We shall see that for a 1-site operator, the matrix exponential commutes with
a Kronecker product by the identity:

In [ ]:
expm(np.kron(np.eye(2), sz))

In [ ]:
np.kron(np.eye(2), expm(sz))

#### Non-diagonal matrix

For the off-diagonal matrix $\sigma^x$, the operations still commute across
Kronecker products with the identity:

In [ ]:
expm(np.kron(np.eye(2), sx))

In [ ]:
np.kron(np.eye(2), expm(sx))

All in all, this means we can time-evolve local operators efficiently by
calculating their matrix exponentials locally.

Since in fact $\sigma^x$ is related to $\sigma^z$ by a Hadamard rotation. $T$,
we can compute:
$$
    \exp(\phi \sigma^x)
        = \exp(\phi T \sigma^z T)
        = T \exp(\phi \sigma^z) T
    .
$$
Let's demonstrate:

In [ ]:
hd = np.array([[1, 1], [1, -1]]) * np.sqrt(0.5)

In [ ]:
expm(sx)

In [ ]:
expm(hd @ sz @ hd)

In [ ]:
hd @ expm(sz) @ hd

We might prefer to use the result in the assignment that for a 1-site operator:
$$
    \exp(i t \bm n \cdot \bm \sigma)
        = \cos(t) + i \sin(t) \bm n \cdot \bm \sigma
    .
$$
For imaginary time evolution:
$$
    \exp(\tau \bm n \cdot \bm \sigma)
        = \cos(-i \tau) + i \sin(-i \tau) \bm n \cdot \bm \sigma
        = \cosh(\tau) + \sinh(\tau) \bm n \cdot \bm \sigma
    .
$$

## Action

At this stage, I have layed out the Suzuki-Trotter decomposition as well as how
to represent the gates within each term, so we can go ahead and do TEBD.
We are told to evolve a ferromagnetic state:
$$
    \ket{\psi (t=0)}
        = \ket{\downarrow} \otimes \cdots \otimes \ket{\downarrow}
    .
$$

In [ ]:
# Define components of Hamiltonian
L  = 6
d  = 2
hx = 1.05
hz = 0.5
sx = np.diag([1, 1])[::-1]
sz = np.diag([1,-1])

In [ ]:
down = np.array([1, 0], dtype='float64')
up   = down[::-1]
# build wavefunction in computational basis
psic = 1
for _ in range(L):
    psic = np.kron(psic, down)
phic = 1
for _ in range(L):
    phic = np.kron(phic, up)

# build wavefunction in MPS representation
psim = np.empty(L, dtype='O')
for i in range(psim.size):
    psim[i] = down[:, None]
phim = np.empty(L, dtype='O')
for i in range(phim.size):
    phim[i] = up[:, None]

# store as MPS object
psi = tensor.mps(psic, tensor.bond_rank(1, L, d), L, d, A=psim)
phi = tensor.mps(phic, tensor.bond_rank(1, L, d), L, d, A=phim)

We are asked to calculate the energy of this state which requires a Hamiltonian.

In [ ]:
# Build pieces of Hamiltonian
H_field = np.empty(L, dtype='O')
for i in range(H_field.size):
    H_field[i] = tensor.mpo(L, d)
    H_field[i][i] = -(hx * sx + hz * sz)
H_even = np.empty(L//2, dtype='O')
for i in range(H_even.size):
    H_even[i] = tensor.mpo(L, d)
    H_even[i][2*i] = -sz
    H_even[i][2*i+1] = sz
H_odd = np.empty(L//2 - 1 + L%2, dtype='O')
for i in range(H_odd.size):
    H_odd[i] = tensor.mpo(L, d)
    H_odd[i][2*i + 1] = -sz
    H_odd[i][2*(i+1)] = sz
H_full = np.array([*H_field, *H_even, *H_odd], dtype='O')

In [ ]:
sum(psi.expval(e) for e in H_full)

In [ ]:
sum(phi.expval(e) for e in H_full)

In [ ]:
np.inner(psic, models.tfim_z.H_vec(psic, L, hx, 'o', hz))

In [ ]:
np.inner(phic, models.tfim_z.H_vec(phic, L, hx, 'o', hz))

### Brenden's approach

This is Brenden's approach, which seems to have the property of contracting a
bond index first, which will probably save storage because it only applies gates
to the physical indices that have no bond indices themselves.

We will first exponentiate each group in the Hamiltonian

In [ ]:
dtau = 0.001

# Verify formula
hn = np.sqrt(hx ** 2 + hz ** 2)
np.allclose(
    expm(dtau * -(hx * sx + hz * sz)),
    np.cosh(dtau * hn) * np.eye(d) - np.sinh(dtau * hn) * (hx * sx + hz * sz) / hn
)

In [ ]:
# Construct propagators
U_field = tensor.mpo(L, d)
for i, e in enumerate(H_field):
    U_field[i] = expm(dtau*e[i])
    
U_even = np.empty(L//2 + L%2, dtype='O')
for i, e in enumerate(H_even):
    U_even[i] = expm(np.kron(*[o for o in e if isinstance(o, np.ndarray)]))

U_odd = np.empty(L//2 + 1 - L%2, dtype='O')
for i, e in enumerate(H_odd):
    U_odd[i+1] = expm(np.kron(*[o for o in e if isinstance(o, np.ndarray)]))

In [ ]:
# choose max rank
chi_max = 5
rank = tensor.bond_rank(chi_max, L, d)
# init
Nstp = 100
energies = np.empty(Nstp + 1, dtype='float64')
energies[0] = sum(phi.expval(e) for e in H_full)
step = [ e for e in phim ]
# TEBD pattern
for j in range(Nstp):
    ## Apply 1-local field terms
    for i, e in enumerate(U_field):
        if i == 0:
            prev_bond_dim = 1
        else:
            prev_bond_dim = step[i-1].shape[1]
        step[i] = np.kron(e, np.eye(prev_bond_dim)) @ step[i]
    # let a, b, c be bond indices and d the physical index
    new = []
    ### contract odd bond indices and apply odd operators
    iterator = enumerate(step)
    iterbtor = iter(U_odd)
    for i, e in iterator:
        if ((i > 0) and (i + 2 < L)):
            if i == 0:
                prev_bond_dim = 1
            else:
                prev_bond_dim = step[i-1].shape[1]
            new.append(
                # apply odd operator to (dd) index
                (np.kron(next(iterbtor), np.eye(prev_bond_dim))
                # Contract bond index: (ad, b), (bd, c) -> (add, c)
                @ (np.kron(np.eye(d), step[i]) @ step[i+1])).reshape(
                # regroup physical indices by row/column: (add,c) -> (ad, dc)
                    (step[i].shape[0], d*step[i+1].shape[1]), order='F'
                )
            )
            next(iterator)
        else:
            new.append(step[i])
            next(iterbtor)
    ### svd on regrouped indices to separate physical indices
    rep = []
    for i, e in enumerate(new):
        if ((i > 0) and (2*i + 1 < L)):
            # separate physical indices by a bond: (ad, dc) -> (ad, b), (b, dc)
            u,s,vh= np.linalg.svd(e, full_matrices=False)
            # use left-canonical form
            rep.append(u)
            # send a physical index to the row index (b, dc) -> (bd, c)
            rep.append(
                (s[:, None] * vh).reshape(
                    (vh.shape[0]*d, vh.shape[1]//d), order='F'
                )
            )
        else:
            rep.append(e)
    ### contract bond indices to form even pairs and apply even operator AGAIN
    neext = []
    iterator = enumerate(rep)
    iterbtor = iter(U_even)
    for i, e in iterator:
        if ((i + 1) < len(rep)):
            if i == 0:
                prev_bond_dim = 1
            else:
                prev_bond_dim = rep[i-1].shape[1]
            neext.append(
                # apply even operator to (dd) index
                (np.kron(next(iterbtor), np.eye(prev_bond_dim))
                # contract bond index: (ad, b), (bd, c) -> (add, c)
                @ (np.kron(np.eye(d), rep[i]) @ rep[i+1])).reshape(
                # regroup physical indices by row/column: (add,c) -> (ad, dc)
                    (rep[i].shape[0], d*rep[i+1].shape[1]), order='F'
                )
            )
            next(iterator)
        else:
            neext.append(rep[i])
            next(iterbtor)
    ### Separate physical indices AGAIN
    last = []
    for i, e in enumerate(neext):
        if (2*i + 1 < L):
            # separate physical indices with bond indices: (ad, dc) -> (ad, b), (b, dc)
            u,s,vh = np.linalg.svd(e, full_matrices=False)
            # use left-canonical form
            last.append(u)
            # send a physical index to the row index (b, dc) -> (bd, c)
            last.append(
                (s[:, None] * vh).reshape(
                    (vh.shape[0]*d, vh.shape[1]//d), order='F'
                )
            )
        else:
            last.append(e)
    ### return to mps form, left canonical, with truncation
    # Sweep from left to right, shifting the orthogonality center
    for i in range(len(last) - 1):
        # contract virtual index: (ad, b), (bd, c) -> (add, c)
        sites = np.kron(np.eye(d), last[i]) @ last[i+1]
        # separate physical indices by row, column: (add, c) -> (ad, dc)
        sites = sites.reshape((sites.shape[0]//d, sites.shape[1]*d), order='F')
        # expand virtual index (ad, dc) -> (ad, b), (b, dc)
        u,s,vh = np.linalg.svd(sites, full_matrices=False)
        ### TODO: truncate the shapes to arbitrary ranks
        # move orthogonality center to the right
        last[i] = u[:d*rank(i), :rank(i+1)]
        # truncate here and renormalize Schmidt values
        last[i+1] = (
            s[:rank(i+1), None] * vh[:rank(i+1), :] / np.linalg.norm(s[:rank(i+1)])
        )
        # restore column physical index to row: (b, dc) -> (bd, c)
        last[i+1] = last[i+1].reshape(
            (last[i+1].shape[0]*d, last[i+1].shape[1]//d), order='F'
        )
    step = last
    # measure energy
    dummy = np.zeros(2**L)
    dummy[0] = 1
    test = tensor.mps(dummy, rank, L, d, A=step)
    energies[j+1] = sum(test.expval(e) for e in H_full)

In [ ]:
# step
# new
# rep
# neext
# last
# energies
test.contract_bonds()
# phi.contract_bonds()
# psi.contract_bonds()

### MPO approach

Here we use the approach I described that was relayed to me by Gil.
There is also an excellent explanation of this on the Tensor Ne

## Extra reading

You might also be interested in reading
- [Feynman on simulation](https://link.springer.com/article/10.1007/BF02650179)
- [Quantum computation by a Caltech sophomore](https://arxiv.org/abs/2103.12783)